# Large Donuts - SITCOM-476

This notebook contains the execution of the Large Donut SITCOM-476 test described in https://jira.lsstcorp.org/browse/SITCOM-476, expected to run during the 2022-10A AuxTel Observing Run. 
                
This notebook is organized in  sections:

    1.1 Setup
    1.2 Conditions assessment
    1.3. Choose target from list of targets in ticket. 
    1.4. CWFS nearby
    1.5. Slew to target object.
    1.6  Perform the data acquisition sequence. 

-----
## Setup

### Import libraries

In [1]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
from astropy.time import Time
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

### Setting up logger

In [2]:
logger = logging.getLogger("SITCOM-476")
logger.level = logging.DEBUG

### Getting unique index for script

In [3]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

### Instantiate CWFS Script

In [4]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

### Forward ATCS and LATISS

In [5]:
atcs = script.atcs
latiss = script.latiss

### Set up script log level

In [6]:
script.log.level = logging.DEBUG

### Write start info into EFD

In [7]:
script.log.info(f'START- SITCOM-476 Large Donuts -- at {Time.now()}')

----
## Assess that conditions meet criteria
Acceptable conditions to avoid boiling in the image: not at the beginning of the night, and seeing less than ~1.2 arcseconds.

----
## Choose target: Target needs to be above 70 degrees to safely move the hexapod the requested amount z_offset=+-7.5 mm. See plots attached by Eske to select the target, depending on the time of the night. https://jira.lsstcorp.org/browse/SITCOM-476

### Declare target and filter

In [13]:
target = "HD223352"

In [9]:
filter_to_use = 'SDSSr_65mm'

------
## CWFS
A CWFS is recommended just before the test to ensure the system is in focus. Based on the target, choose a nearby target to perform CWFS on. 

### Declare CWFS target 

Query for a source around the same area of the sky as the target source.

In [10]:
cwfs_az = 90
cwfs_el = 60
cwfs_mag_limit = 8

In [11]:
cwfs_target = await script.atcs.find_target(az=cwfs_az, el=cwfs_el, mag_limit=cwfs_mag_limit)
logger.info(f'Target for CWFS with magnitude limit {cwfs_mag_limit} is {cwfs_target}')

### Slew to the CWFS target

In [12]:
await script.atcs.slew_object(cwfs_target, rot_type=RotType.PhysicalSky)

(<ICRS Coordinate: (ra, dec) in deg
     (2.23048958, -26.22364861)>,
 <Angle 0. deg>)

Uncomment the following line to take a snapshot to verify the target is positioned as expected 

In [14]:
exposure = await latiss.take_acq(
         exptime=5, n=1, filter=filter_to_use, grating='empty_1', reason='Acquisition', program="SITCOM-476")
logger.info(f'Acquisition exposure is {exposure}')

To offset the telescope and center the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

### Set up configuration

In [15]:
configuration = yaml.safe_dump({"filter": filter_to_use, 
                                "grating": 'empty_1',
                                "exposure_time": 20,
                                "program" : "SITCOM-476"})

The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [16]:
await script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [17]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [18]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script.

In [19]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

### Stop tracking: If required, then uncomment and use the following cell to stop the telescope from tracking, but you will lose your acquisition.

In [ ]:
# await script.atcs.stop_tracking()

-------
## Slew to target object

### Confirm that the target is correct

In [20]:
logger.info(f'Target is {target}')

### Slew to the target object

In [21]:
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

(<ICRS Coordinate: (ra, dec) in deg
     (357.23144375, -28.13027056)>,
 <Angle 0. deg>)

In [22]:
exposure = await latiss.take_acq(
         exptime=1, n=1, filter=filter_to_use, grating='empty_1', reason='Acquisition', program="SITCOM-476")
logger.info(f'Acquisition exposure is {exposure}')

To offset the telescope and move the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

----
## Data Acquisition Sequence

### Declare z offset 

In [23]:
z_offset = 7.5

### Check focus summary

In [24]:
original_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
logger.info(f'Original focus offset is \n {original_focus_offset}')

### Going extra focus with +z_offset

In [25]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=z_offset)

###  Check signal level: Take an image and make sure few thousand counts per pixel in the illuminated annulus are present. 

In [26]:
saturation_test = await latiss.take_engtest(
        exptime=30, n=1, filter=filter_to_use, grating='empty_1', reason='Exposure_Time_Test', program = "SITCOM-476")
logger.info(f'Saturation test exposure is {saturation_test}')

### Data Acquisition Extra Focus

In [27]:
extra_images = await latiss.take_focus(
    exptime=30, n=5, filter=filter_to_use, grating='empty_1', reason='Large_Donuts_extra', program ="SITCOM-476")
logger.info(f'Extra-Focus images are {extra_images}')

### Going intra focus with -z_offset

In [28]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=-2*z_offset)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1070028796, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Timed out when waiting for correction loop to apply offsets during offset command')

In [29]:
# Moving 15mm at one time takes too long so ATAOS timed out and faulted
# In the future, make this move in two steps.
await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

### Data Acquisition Intra Focus

In [30]:
intra_images = await latiss.take_focus(
    exptime=30, n=5, filter=filter_to_use, grating='empty_1', reason='Large_Donuts_intra', program ="SITCOM-476")
logger.info(f'Intra-Focus images are {intra_images}')

### Move hexapod back to in-focus (zero-offset) position

In [31]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=z_offset)

### Confirm the focus offset is back to where it was and wrap up. 

In [36]:
current_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
logger.info(current_focus_offset)

In [ ]:
# Original and current focus offset's user comparison 

In [37]:
logger.info(f'Current {current_focus_offset.userApplied:0.3f} mm focus vs. original {original_focus_offset.userApplied:0.3f} mm focus offsets')

In [38]:
script.log.info(f'END- SITCOM-476 Large Donuts -- at {Time.now()}')